<a href="https://colab.research.google.com/github/virajsazzala/image-classifier/blob/main/cats_and_dogsv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data

In [36]:
# unzip dataset

import zipfile

local_zip = 'catsanddogs.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./catsanddogs')
zip_ref.close()

In [37]:
# labelling images with ImageDataGenerator

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)
train_generator = train_datagen.flow_from_directory(
    './catsanddogs/catsanddogs/',
    target_size = (150, 150),
    batch_size = 10,
    class_mode = 'binary')

Found 697 images belonging to 2 classes.


## Model

In [38]:
# create callback class

import tensorflow as tf

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if logs.get('accuracy') is not None and logs.get('accuracy') > 0.999:
      print('\nReached 95.0% accuracy, Cancelling training!')
      self.model.stop_training = True

In [39]:
# model

from tensorflow.keras import optimizers, losses
from tensorflow.keras.optimizers import RMSprop

callbacks = myCallback()

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),

  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.RMSprop(learning_rate=0.001),
    metrics=['accuracy'])

history = model.fit(
    train_generator,
    epochs=20,
    callbacks=[callbacks])


Epoch 1/20
70/70 [==============================] - 5s 54ms/step - loss: 1.2654 - accuracy: 0.4950
Epoch 2/20
70/70 [==============================] - 4s 54ms/step - loss: 0.7068 - accuracy: 0.5624
Epoch 3/20
70/70 [==============================] - 5s 71ms/step - loss: 0.6845 - accuracy: 0.5811
Epoch 4/20
70/70 [==============================] - 4s 60ms/step - loss: 0.6514 - accuracy: 0.6471
Epoch 5/20
70/70 [==============================] - 4s 53ms/step - loss: 0.5825 - accuracy: 0.7088
Epoch 6/20
70/70 [==============================] - 4s 60ms/step - loss: 0.5165 - accuracy: 0.7489
Epoch 7/20
70/70 [==============================] - 4s 59ms/step - loss: 0.4298 - accuracy: 0.8063
Epoch 8/20
70/70 [==============================] - 4s 55ms/step - loss: 0.3162 - accuracy: 0.8637
Epoch 9/20
70/70 [==============================] - 4s 60ms/step - loss: 0.2421 - accuracy: 0.9082
Epoch 10/20
70/70 [==============================] - 5s 70ms/step - loss: 0.1780 - accuracy: 0.9369
Epoch 11/

## Test

In [53]:
# upload files to test

import numpy as np
from google.colab import files
from tensorflow.keras.utils import img_to_array, load_img

uploaded = files.upload()

for fn in uploaded.keys():
  path = '/content/' + fn
  img = load_img(path, target_size=(150, 150))
  x = img_to_array(img)
  x /= 255
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images)
  print(classes[0])

  if classes[0] > 0.5:
    print(fn + " is a dog")
  else:
    print(fn + " is a cat")

Saving cat1.jpg to cat1 (2).jpg
1/1 [==============================] - 0s 17ms/step
[0.592176]
cat1 (2).jpg is a dog
